# Transformar imágenes
## Transparencia de imágenes

In [ ]:
import os
import qrcode
from PIL import Image, ImageDraw

ruta = os.path.dirname(os.getcwd())

In [16]:
def make_white_transparent(image_path, output_path):
    """
    Makes the white color in a PNG image transparent.

    Args:
        image_path: Path to the input PNG image.
        output_path: Path to save the modified PNG image.
    """
    img = Image.open(image_path).convert("RGBA")
    datas = img.getdata()

    new_data = []
    for item in datas:
        if item[0] > 250 and item[1] > 250 and item[2] > 250:
            new_data.append((255, 255, 255, 0))  # Make it transparent
        else:
            new_data.append(item)

    img.putdata(new_data)
    img.save(output_path,format='ICO')

In [ ]:
input_image = ruta+"/image/Logo.png"  # Replace with your image path
output_image = ruta+"/image/Logo_transparent.png"
make_white_transparent(input_image, output_image)

## QR Code with logo

In [40]:
def create_qr_with_logo(data_to_encode, logo_path, output_path="qr_with_logo.png", qr_color="black", bg_color="white", box_size=10, border=4):
    """
    Generates a QR code with a logo embedded in the center.

    Args:
        data_to_encode (str): The data (e.g., URL, text) to encode in the QR code.
        logo_path (str): The file path to the logo image.
        output_path (str, optional): The file path to save the generated QR code. Defaults to "qr_with_logo.png".
        qr_color (str, optional): Color of the QR code modules. Defaults to "black".
        bg_color (str, optional): Background color of the QR code. Defaults to "white".
        box_size (int, optional): Controls how many pixels each “box” of the QR code is. Defaults to 10.
        border (int, optional): Controls how many boxes thick the border should be (the default is 4, which is the minimum according to the specs).
    """
    try:
        # --- Step 1: Generate the QR Code ---
        # Configure the QR code
        qr = qrcode.QRCode(
            version=None,  # Let the library automatically determine the version
            error_correction=qrcode.constants.ERROR_CORRECT_H,  # High error correction to accommodate the logo
            box_size=box_size,
            border=border,
        )
        qr.add_data(data_to_encode)
        qr.make(fit=True)

        # Create an image from the QR Code instance
        img_qr = qr.make_image(fill_color=qr_color, back_color=bg_color).convert('RGBA')
        qr_width, qr_height = img_qr.size

        # --- Step 2: Load and Prepare the Logo ---
        try:
            logo = Image.open(logo_path)
        except FileNotFoundError:
            print(f"Error: Logo file not found at {logo_path}")
            return
        except Exception as e:
            print(f"Error opening logo file: {e}")
            return

        # Calculate the maximum logo size (e.g., 25-30% of the QR code's width)
        # This is to ensure the QR code remains scannable
        logo_max_size_w = int(qr_width * 0.3)
        logo_max_size_h = int(qr_height * 0.3)
        logo_max_size = min(logo_max_size_w, logo_max_size_h) # Use the smaller dimension to maintain aspect ratio

        # Resize the logo
        logo.thumbnail((logo_max_size, logo_max_size), Image.Resampling.LANCZOS) # Use LANCZOS for high-quality downscaling
        logo_w, logo_h = logo.size

        # --- Step 3: Position and Paste the Logo onto the QR Code ---
        # Calculate the position to paste the logo (center of the QR code)
        pos_x = (qr_width - logo_w) // 2
        pos_y = (qr_height - logo_h) // 2

        # Create a mask for the logo if it has transparency
        # This helps in pasting the logo smoothly
        if logo.mode == 'RGBA':
            mask = logo
        else:
            # If no alpha channel, create a solid mask or convert logo to RGBA
            # For simplicity, we'll assume a rectangular logo here.
            # For non-rectangular logos without alpha, more advanced masking might be needed.
            logo = logo.convert('RGBA')
            mask = logo

        # Create a white background for the logo area on the QR code
        # This helps the logo stand out and can improve scannability if the logo is dark
        # The background should be slightly larger than the logo
        bg_offset = 2 # Small padding around the logo
        bg_draw_pos_x = pos_x - bg_offset
        bg_draw_pos_y = pos_y - bg_offset
        bg_draw_width = logo_w + (bg_offset * 2)
        bg_draw_height = logo_h + (bg_offset * 2)

        # Create a drawing object
        draw = ImageDraw.Draw(img_qr)
        # Draw a white rectangle (or bg_color) behind where the logo will be placed
        draw.rectangle(
            (bg_draw_pos_x, bg_draw_pos_y, bg_draw_pos_x + bg_draw_width, bg_draw_pos_y + bg_draw_height),
            fill=bg_color
        )

        # Paste the logo onto the QR code
        img_qr.paste(logo, (pos_x, pos_y), mask)

        # --- Step 4: Save the Final Image ---
        img_qr.save(output_path)
        print(f"Successfully created QR code with logo: {output_path}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [41]:
my_data = "https://forms.gle/M9gJBGUCFeE3FPt39"

my_logo_path = ruta+"/image/Logo_transparent.png" 

create_qr_with_logo(
    data_to_encode=my_data,
    logo_path=my_logo_path,
    qr_color="#163D40", # Example: dark blue QR code
    # bg_color="#3b82f6" # Example: light yellow background
    output_path=ruta+"/image/qr_with_logo.png"
)

Successfully created QR code with logo: c:\Users\ismaelaguilera\Documents\html-basic-project/image/qr_with_logo.png


In [11]:
import os
print(os.path.dirname(os.getcwd()))


c:\Users\ismaelaguilera\Documents\html-basic-project
